## About

In this notebook we prepare a simple solution for the [kaggle challenge on higgs.](https://inclass.kaggle.com/c/mlhep-2016-higgs-detection)

In [76]:
%matplotlib inline

In [77]:
import matplotlib.pyplot as plt

import pandas
import numpy as np

### Download data

In [ ]:
!cd datasets; wget -O public_train_10000.root -nc --no-check-certificate https://2016.mlhep.yandex.net/data/higgs/public_train_10000.root

In [ ]:
# you can download training sample with 100000 available events
# uncomment the below row
!cd datasets; wget -O public_train_100000.root -nc --no-check-certificate https://2016.mlhep.yandex.net/data/higgs/public_train_100000.root

In [ ]:
!cd datasets; wget -O public_test.root -nc --no-check-certificate https://2016.mlhep.yandex.net/data/higgs/public_test.root

In [154]:
!cd datasets; wget -O test_sorted.root -nc --no-check-certificate https://transfer.sh/Bvy4H/test-sorted.root

wget: /root/miniconda/envs/rep_py2/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /root/miniconda/envs/rep_py2/lib/libssl.so.1.0.0: no version information available (required by wget)
--2016-06-23 09:56:40--  https://transfer.sh/Bvy4H/test-sorted.root
Resolving transfer.sh (transfer.sh)... 64:ff9b::3433:a179, 64:ff9b::3412:7e25
Connecting to transfer.sh (transfer.sh)|64:ff9b::3433:a179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92696730 (88M)
Saving to: 'test_sorted.root'

100%[======================================>] 92,696,730  13.2MB/s   in 10s    

2016-06-23 09:56:50 (8.64 MB/s) - 'test_sorted.root' saved [92696730/92696730]



### Read the smallest part of training file and test file

In [158]:
import root_numpy
#data = pandas.DataFrame(root_numpy.root2array('datasets/public_train_100000.root'))
data = pandas.DataFrame(root_numpy.root2array('datasets/public_train_100000_sorted.root'))
test = pandas.DataFrame(root_numpy.root2array('datasets/test_sorted.root'))
data.head()

,event_id,target,lepton_pt,lepton_eta,lepton_phi,mem_pt,mem_phi,jet1_pt,jet1_eta,jet1_phi,...,jet4_phi,jet4_btag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb,__index__
0,1000001,1,34.750568,0.787025,1.898891,20.862434,-2.622998,79.948036,0.877472,-0.256736,...,2.631595,2.000023,81.724449,189.583145,80.118317,170.086075,91.128204,298.468781,374.685760,0
1,1000002,1,54.250927,-1.057915,2.310697,51.167873,2.545749,71.681404,-1.139118,-1.300325,...,-0.737298,0.000000,65.837746,201.096756,83.321556,208.039688,67.118484,287.363983,527.247559,1
2,1000003,1,47.746025,-0.783184,2.660325,68.165527,-1.700790,118.880913,-0.211263,1.326902,...,-0.626912,0.000000,69.316925,156.334732,95.307602,149.089005,130.389206,237.879318,336.058838,2
3,1000004,0,45.950066,1.613817,0.964722,39.302082,-0.075989,84.307426,0.465748,2.287783,...,-0.850500,0.000000,71.032066,182.341537,81.941925,164.411148,93.709511,237.900055,392.807831,3
4,1000005,0,44.409187,-0.228907,-1.837974,49.886654,0.156533,138.741928,0.293522,1.391425,...,-1.192486,0.000000,122.030174,288.594086,84.386459,150.299744,69.818291,435.990356,533.977905,4


In [157]:
test.shape

(1000000, 30)

### Define training features

Exclude `event_id`, `target` from the features set

In [165]:
features = list(set(data.columns) - {'event_id', 'target', '__index__',
                                     'jet1_btag', 'jet2_btag', 'jet3_btag', 'jet4_btag'})
features

['jet3_pt',
 'jet3_eta',
 'm_jjj',
 'mem_phi',
 'jet1_pt',
 'jet4_phi',
 'jet1_phi',
 'jet2_eta',
 'm_jlv',
 'm_wbb',
 'jet4_pt',
 'jet2_pt',
 'm_jj',
 'm_wwbb',
 'jet2_phi',
 'lepton_phi',
 'm_bb',
 'm_lv',
 'jet4_eta',
 'lepton_pt',
 'mem_pt',
 'lepton_eta',
 'jet3_phi',
 'jet1_eta']

### Prepare high-level features for training

In [166]:
from itertools import combinations
new_data = data.copy()
new_test = test.copy()
new_features = features[:]
for m1, m2 in combinations([i for i in data.columns if 'm_' in i and '?' not in i and 'mem' not in i], 2):
    if len(m1) < len(m2):
        m1, m2 = m2, m1
    new_data['?'+m1+'/'+m2] = data[m1].values / data[m2].values
    new_test['?'+m1+'/'+m2] = test[m1].values / test[m2].values
    new_features.append('?'+m1+'/'+m2)
new_data['sum_pt'] = data.jet1_pt + data.jet2_pt + data.jet3_pt + data.jet4_pt + data.lepton_pt + data.mem_pt
new_test['sum_pt'] = test.jet1_pt + test.jet2_pt + test.jet3_pt + test.jet4_pt + test.lepton_pt + test.mem_pt
new_features.append('sum_pt')
new_data['sum_jet_pt'] = data.jet1_pt + data.jet2_pt + data.jet3_pt + data.jet4_pt
new_test['sum_jet_pt'] = test.jet1_pt + test.jet2_pt + test.jet3_pt + test.jet4_pt
new_features.append('sum_jet_pt')

In [168]:
new_features

['jet3_pt',
 'jet3_eta',
 'm_jjj',
 'mem_phi',
 'jet1_pt',
 'jet4_phi',
 'jet1_phi',
 'jet2_eta',
 'm_jlv',
 'm_wbb',
 'jet4_pt',
 'jet2_pt',
 'm_jj',
 'm_wwbb',
 'jet2_phi',
 'lepton_phi',
 'm_bb',
 'm_lv',
 'jet4_eta',
 'lepton_pt',
 'mem_pt',
 'lepton_eta',
 'jet3_phi',
 'jet1_eta',
 '?m_jjj/m_jj',
 '?m_jj/m_lv',
 '?m_jlv/m_jj',
 '?m_jj/m_bb',
 '?m_wbb/m_jj',
 '?m_wwbb/m_jj',
 '?m_jjj/m_lv',
 '?m_jjj/m_jlv',
 '?m_jjj/m_bb',
 '?m_jjj/m_wbb',
 '?m_wwbb/m_jjj',
 '?m_jlv/m_lv',
 '?m_lv/m_bb',
 '?m_wbb/m_lv',
 '?m_wwbb/m_lv',
 '?m_jlv/m_bb',
 '?m_jlv/m_wbb',
 '?m_wwbb/m_jlv',
 '?m_wbb/m_bb',
 '?m_wwbb/m_bb',
 '?m_wwbb/m_wbb',
 'sum_pt',
 'sum_jet_pt']

### Plot histograms for each high-level feature

In [ ]:
hist_params = {'normed': True, 'bins': 60, 'alpha': 0.4}
# create the figure
plt.figure(figsize=(16, 25))
for n, feature in enumerate(high_level_features):
    # add sub plot on our figure
    plt.subplot(len(features) // 5 + 1, 3, n+1)
    # define range for histograms by cutting 1% of data from both ends
    min_value, max_value = numpy.percentile(data[feature], [1, 99])
    plt.hist(data.ix[data.target.values == 0, feature].values, range=(min_value, max_value), 
             label='class 0', **hist_params)
    plt.hist(data.ix[data.target.values == 1, feature].values, range=(min_value, max_value), 
             label='class 1', **hist_params)
    plt.legend(loc='best')
    plt.title(feature)

In [137]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC

def get_validated_trained(fitter, data, features, part):
    training_data, validation_data = train_test_split(data, random_state=3747824, train_size=part)
    fitter.fit(training_data[features], training_data.target)
    results = fitter.predict_proba(validation_data[features])
    print 'Validation:', roc_auc_score(validation_data.target, results[:, 1])

def get_full_trained(fitter, data, features):
    fitter.fit(data[features], data.target)

def get_result(fitter, test, features):
    return fitter.predict_proba(test[features])[:, 1]

In [106]:
from sknn.mlp import Classifier, Layer, Convolution
import sys
import logging

logging.basicConfig(
    stream=sys.stdout
)
logger = logging.getLogger('sknn')
logger.setLevel(logging.DEBUG)

fitter = Pipeline([
        ('min/max scaler', MinMaxScaler(feature_range=(0.0, 1.0))),
        ('neural network', Classifier(
                layers=[
                    Convolution("Sigmoid", channels=8, kernel_shape=(3,3)),
                    Layer("Rectifier", units=300),
                    Layer("Rectifier", units=300),
                    Layer("Softmax")
                ],
                n_iter=50,
                random_state=274734,
                learning_rate=0.21,
                learning_momentum=0.999,
                verbose=True,
                batch_size=10,
                regularize='L2',
                weight_decay=0.00001
    ))])

cur_features = features
get_validated_trained(fitter, data, cur_features, 0.6)

AssertionError: Input array is not in image shape, and could not assume a square.

In [178]:
from rep.estimators import XGBoostClassifier
from sklearn import tree
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV

tuned_parameters = {
    'max_depth': [8],
    'n_estimators': [300],
    'eta': [0.02, 0.03, 0.04, 0.05, 0.06]
}
fitter = GridSearchCV(XGBoostClassifier(
        #max_depth=13,
        #n_estimators=300,
), tuned_parameters, scoring='roc_auc', verbose=100500)
get_validated_trained(fitter, new_data.astype('float'), new_features, 0.7)
#get_full_trained(fitter, data, features)
print fitter.best_params_

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] n_estimators=300, eta=0.02, max_depth=8 .........................
[CV]  n_estimators=300, eta=0.02, max_depth=8, score=0.802537 -  40.9s
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:   40.9s
[CV] n_estimators=300, eta=0.02, max_depth=8 .........................
[CV]  n_estimators=300, eta=0.02, max_depth=8, score=0.808590 -  40.3s
[Parallel(n_jobs=1)]: Done   2 jobs       | elapsed:  1.4min
[CV] n_estimators=300, eta=0.02, max_depth=8 .........................
[CV]  n_estimators=300, eta=0.02, max_depth=8, score=0.804838 -  40.9s
[Parallel(n_jobs=1)]: Done   3 jobs       | elapsed:  2.0min
[CV] n_estimators=300, eta=0.03, max_depth=8 .........................
[CV]  n_estimators=300, eta=0.03, max_depth=8, score=0.802397 -  48.1s
[Parallel(n_jobs=1)]: Done   4 jobs       | elapsed:  2.8min
[CV] n_estimators=300, eta=0.03, max_depth=8 .........................
[CV]  n_estimators=300, eta=0.03, max_depth=8, score=0.80

In [223]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

tuned_parameters = {
    #'alpha': [0.0001],
    #'power_t': [0.1, 0.5, 0.7],
    #'n_iter': [42, 66, 77, 100]
    'n_estimators': [700],
    'learning_rate': [0.2, 0.3]
}
boost_fitter = AdaBoostClassifier()
grid_fitter = GridSearchCV(boost_fitter, tuned_parameters,
                              scoring='roc_auc', verbose=100500)
fitter = Pipeline([
        ('min/max scaler', MinMaxScaler(feature_range=(0.0, 1.0))),
        ('grid', grid_fitter)])
get_validated_trained(fitter, new_data.astype('float'), new_features, 0.7)
#get_full_trained(fitter, data, features)
print grid_fitter.best_params_

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] n_estimators=700, learning_rate=0.2 .............................
[CV] .... n_estimators=700, learning_rate=0.2, score=0.777832 - 3.5min
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:  3.5min
[CV] n_estimators=700, learning_rate=0.2 .............................
[CV] .... n_estimators=700, learning_rate=0.2, score=0.785885 - 3.7min
[Parallel(n_jobs=1)]: Done   2 jobs       | elapsed:  7.3min
[CV] n_estimators=700, learning_rate=0.2 .............................
[CV] .... n_estimators=700, learning_rate=0.2, score=0.783012 - 3.5min
[Parallel(n_jobs=1)]: Done   3 jobs       | elapsed: 10.8min
[CV] n_estimators=700, learning_rate=0.3 .............................
[CV] .... n_estimators=700, learning_rate=0.3, score=0.778066 - 3.5min
[Parallel(n_jobs=1)]: Done   4 jobs       | elapsed: 14.3min
[CV] n_estimators=700, learning_rate=0.3 .............................
[CV] .... n_estimators=700, learning_rate=0.3, score=0.786

In [179]:
fitter = XGBoostClassifier(
    max_depth=8,
    n_estimators=300,
    eta=0.02
)
#get_validated_trained(fitter, data.astype('float'), features, 0.7)
get_full_trained(fitter, new_data.astype('float'), new_features)

## Prepare submission to kaggle

In [180]:
# predict test sample
kaggle_proba = get_result(fitter, new_test.astype('float'), new_features)
kaggle_ids = test.event_id

In [181]:
from IPython.display import FileLink
def create_solution(ids, proba, filename='xgboost.csv'):
    """saves predictions to file and provides a link for downloading """
    pandas.DataFrame({'event_id': ids, 'prediction': proba}).to_csv('datasets/{}'.format(filename), index=False)
    return FileLink('datasets/{}'.format(filename))
    
create_solution(kaggle_ids.astype(np.int32), kaggle_proba)

/notebooks/higgs_kaggle/datasets/xgboost.csv